In [2]:
import pandas as pd
from pathlib import Path

import sys
sys.path.append('../src')
from utils.duckdb_utils import connect_duckdb
from utils.duckdb_utils import run_sql_file
# import utils.site_list as sl

In [3]:
with connect_duckdb() as con:
    tables = con.execute("SHOW TABLES").fetchall()
    table_names = [table[0]for table in tables]

    print(table_names)

['daily_observations', 'parameter', 'site', 'site_parameter', 'vw_nwis_annual_stats_local', 'vw_nwis_daily_stats_local', 'vw_nwis_iv_local']


In [13]:
query = """
    SELECT hydro_area_nm, site_nm, year, max_value, max_date
    FROM vw_nwis_annual_stats_local
    WHERE parameter_cd = '00060'
    order by year DESC, hydro_area_nm, site_nm
    """
query = "SELECT * FROM parameter"

with connect_duckdb() as con:
    result = con.execute(query).df()
    print(result.head())

   parameter_id parameter_cd                 parameter_nm parameter_dsc  \
0             1        00060                    Discharge          None   
1             2        00010            Water temperature          None   
2             3        62614  Reservoir surface elevation          None   
3             4           15                 Bank storage          None   
4             5           29                       Inflow          None   

  unit_cd                unit_nm                  create_ts  \
0     cfs  Cubic feet per second 2025-06-17 17:28:46.429526   
1       C        Degrees Celsius 2025-06-17 17:28:46.429526   
2      ft                   Feet 2025-06-17 17:28:46.429526   
3      af              Acre-feet 2025-06-17 17:28:46.429526   
4     cfs  Cubic feet per second 2025-06-17 17:28:46.429526   

                   update_ts  
0 2025-06-17 17:28:46.429880  
1 2025-06-17 17:28:46.429880  
2 2025-06-17 17:28:46.429880  
3 2025-06-17 17:28:46.429880  
4 2025-06-17 17

In [32]:
nwis_params = pd.read_csv("../artifacts/USGS_parameter_codes.tsv", sep="\t", dtype=str)
nwis_params = nwis_params.loc[~nwis_params['parameter_cd'].isin(['00060', '00010'])].reset_index(drop=True)
nwis_params['parameter_id'] = range(23, len(nwis_params) + 23)
nwis_params['create_ts'] = pd.Timestamp.now()
nwis_params['update_ts'] = pd.Timestamp.now()

In [33]:
column_list = result.columns.tolist()
nwis_params = nwis_params[column_list].copy()

In [34]:
with connect_duckdb() as con:
    con.register('nwis_params', nwis_params)
    con.execute("INSERT INTO parameter SELECT * FROM nwis_params")

In [4]:
query = "SELECT * FROM site"
query = "SELECT * FROM parameter"

In [22]:
run_sql_file(Path('../db/views.sql'))

In [12]:
with connect_duckdb() as con:
    result = con.execute(query).df()
print(result.head())

   parameter_id parameter_cd                 parameter_nm parameter_dsc  \
0             1           60                    Discharge          None   
1             2           10            Water temperature          None   
2             3        62614  Reservoir surface elevation          None   
3             4           15                 Bank storage          None   
4             5           29                       Inflow          None   

  unit_cd                unit_nm                  create_ts  \
0     cfs  Cubic feet per second 2025-06-17 17:28:46.429526   
1       C        Degrees Celsius 2025-06-17 17:28:46.429526   
2      ft                   Feet 2025-06-17 17:28:46.429526   
3      af              Acre-feet 2025-06-17 17:28:46.429526   
4     cfs  Cubic feet per second 2025-06-17 17:28:46.429526   

                   update_ts  
0 2025-06-17 17:28:46.429880  
1 2025-06-17 17:28:46.429880  
2 2025-06-17 17:28:46.429880  
3 2025-06-17 17:28:46.429880  
4 2025-06-17 17

In [11]:
param = pd.read_csv('../artifacts/parameters.csv')

with connect_duckdb() as con:
    con.register('param', param)
    con.execute('INSERT INTO parameter SELECT * FROM param')